<b>LING 193 - Class 11<br>
Minimal edit distance in Python</b><br>
Andrew McInnerney<br>
October 5, 2022

#1 Displaying tables
As we build towards a minimal edit distance calculator, it will be really useful to have a function that can display tables for us. Either use the code block below, or copy and paste the function you wrote in class 8.

In [6]:
def print_table(t):
    out = ""
    for row in t:
        for i in row:
            out += str(i)+"\t"
        out += "\n"
    print(out)

# 2 Getting the first row of the table

Our goal today is to write a function `minedit()` that can take two words as input, and return the minimal edit distance table for those words as output. E.g., given the words *Idaho* and *idol*, we should be able to use `minedit('IDAHO', 'IDOL')` to arrive at:

<table>
  <tr></tr>
    <th></th>
    <th>#</th>
    <th>I</th>
    <th>D</th>
    <th>A</th>
    <th>H</th>
    <th>O</th>
  <tr></tr>
  <tr></tr>
    <th>#</th>
    <th>0</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
  <tr></tr>
  <tr></tr>
    <th>I</th>
    <th>1</th>
    <th>0</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
  <tr></tr>
  <tr></tr>
    <th>D</th>
    <th>2</th>
    <th>1</th>
    <th>0</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
  <tr></tr>
  <tr></tr>
    <th>O</th>
    <th>3</th>
    <th>2</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
  <tr></tr>
  <tr></tr>
    <th>L</th>
    <th>4</th>
    <th>3</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
  <tr></tr>
</table>

This is the table read out, though. What `minedit('IDAHO', 'IDOL')` should really give us is a *list*, which we can *interpret* as a table. 

In [ ]:
table = [['', '#', 'I', 'D', 'A', 'H', 'O'], \
         ['#', 0, 1, 2, 3, 4, 5], \
         ['I', 1, 0, 1, 2, 3, 4], \
         ['D', 2, 1, 0, 1, 2, 3], \
         ['O', 3, 2, 1, 2, 3, 2], \
         ['L', 4, 3, 2, 3, 4, 3]] # NOTE: python will ignore a line break if you type \ before it

print_table(table)

	#	I	D	A	H	O	
#	0	1	2	3	4	5	
I	1	0	1	2	3	4	
D	2	1	0	1	2	3	
O	3	2	1	2	3	2	
L	4	3	2	3	4	3	



It will take us several steps to get there. The first thing we will do is figure out how to get the first row in the table. For now, we will ignore the letters printed out as row- and column-labels, and just focus on the numeric cells. The first row for `minedit('IDAHO', 'IDOL')` should be `[0, 1, 2, 3, 4, 5]`.

Let's write a function to generate this row. Call it `firstrow`, and give it one argument `word1`. It should return a list containing the numbers 0 to *n*, with *n* being the number of letters in `word1`.

In [7]:
def firstrow(word):
  letter_list = []
  for index in range(len(word)):
    letter_list.append(index)
  return letter_list


Once you've written this function, run the code block below. You should get these outputs:
```python
[0]
[0, 1, 2, 3, 4]
```

In [ ]:
print(firstrow("A"))
print(firstrow("ABCDE"))

[0]
[0, 1, 2, 3, 4]


If that works, change your code so the argument that `firstrow()` takes is not a word anymore, but a number. This will be useful later:

In [8]:
def firstrow(m):
  return [i for i in range(m+1)]  # This notation is another way of defining a list! If you're interested in learning more, the term to search is "List Comprehension".

Notice how the output here is slightly different:

In [ ]:
print(firstrow(1))
print(firstrow(5))

[0, 1]
[0, 1, 2, 3, 4, 5]


# 3 Calculating the value for substitution
Next, we need a function that does the logic for substitution (moving diagonally in the grid). Remember, substitution where the letters match has a penalty of 0, but has a penalty of 2 otherwise.

Write a function `substitution_penalty()` that takes two letters as input and returns 0 if they match, and 2 otherwise. (E.g., `substitution_penalty("A", "B")` should return 2 as output).

In [9]:
def substitution_penalty(letter1, letter2):
  if letter1 == letter2:
    return 0
  else:
    return 2


If your function is working as intended, running the  code block below should give you these outputs:
```
0
2
```

In [ ]:
print(substitution_penalty("F", "F"))
print(substitution_penalty("A", "Z"))

0
2


# 4 Calculating the next rows
In the end, our `minedit()` function will generate the first row of the table with the `firstrow()` function, and then it will calculate each successive row using another function `nextrow()`, which we will define now.

This function is going to take in three things as input:<br>
&emsp;(i) The row immediately above<br>
&emsp;(ii) The input word<br>
&emsp;(iii) The letter for the current row

More explicitly, suppose we want to make the table for *Idaho* vs *idol*. We start by making the first row with `firstrow()`, giving us this (in the abstract):

<table>
  <tr></tr>
    <th></th>
    <th>#</th>
    <th>I</th>
    <th>D</th>
    <th>A</th>
    <th>H</th>
    <th>O</th>
  <tr></tr>
  <tr></tr>
    <th>#</th>
    <th>0</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
  <tr></tr>
  <tr></tr>
    <th>I</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
  <tr></tr>
    <th>D</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
  <tr></tr>
    <th>O</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
  <tr></tr>
    <th>L</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
</table>

To fill in the *second* row, we'll apply the `nextrow()` function, giving it as arguments:<br>
&emsp;(i) The prior row `[0, 1, 2, 3, 4, 5]`<br>
&emsp;(ii) The input word `'IDAHO'`<br>
&emsp;(iii) The letter for the current row `'I'`

In other words, we will want to apply:
```
nextrow([0,1,2,3,4,5], 'IDAHO', 'I')
```

This function should start by calculating the first cell in the row, then second cell, and on down the line.

The first cell can only be derived via insertion (moving top to bottom), which we can calculate by taking the value of the first cell of the prior row, and adding `1`. In this case, that's `0+1`, giving us `1`:

<table>
  <tr></tr>
    <th></th>
    <th>#</th>
    <th>I</th>
    <th>D</th>
    <th>A</th>
    <th>H</th>
    <th>O</th>
  <tr></tr>
  <tr></tr>
    <th>#</th>
    <th>0</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
  <tr></tr>
  <tr></tr>
    <th>I</th>
    <th>1</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
  <tr></tr>
    <th>D</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
  <tr></tr>
    <th>O</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
  <tr></tr>
    <th>L</th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
    <th></th>
  <tr></tr>
</table>

The rest of the cells in the `'I'` row will be calculated based on the values of previous cells. For each cell, we will need to calculate:<br>
&emsp;(i) The cost of insertion: the value of the cell above + 1<br>
&emsp;(ii) The cost of deletion: the value of the cell to the left + 1<br>
&emsp;(iii) The cost of substitution: the value of the cell diagonally up and left, plus the penalty calculated with the `substitution_penalty()` function.

Of these, we pick the lowest one as the next cell in the row. Then we move on to the next cell, until we've filled in the whole row.

To define the `nextrow()` function, follow these steps:
1. Write the `def` line, with the name `nextrow` as the function label, and variables `priorrow`, `word`, and `letter` as arguments.
2. In the indented definition block now, write a line that starts the new row: Create a list named `row`, and add to it the value of the leftmost cell. Remember, this should be equal to the first value of the *prior* row, plus one. 
3. Next, create a variable called `priorcell`. Set it equal to the first item in `row`. We will keep updating this as we go on down the row, so that `priorcell` will always be equal to the value of the cell we just calculated. 
4. Write a *for*-loop that iterates over the letters in `word1`, and calculates the value of the corresponding cell in the row (hint: use `range()` for this!). Within this loop, you will need to calculate the value of:<br>
&emsp;(i) Insertion<br>
&emsp;(ii) Deletion<br>
&emsp;(iii) Substitution<br>
  - You will need to think about which cell you need to look at to calculate each of these. 
    - E.g., for deletion, you want to look at the cell immediately to the left, which should be `priorcell`. 
    - For insertion and substitution, you want to look at a cell in the prior row.)
  - You might want to give each value its own name, so that you have a variable `insertion`, a variable `deletion`, and a variable `substitution`.
  - Then, use Python's `min()` function to get the minimum value among the three. Update the `priorcell` variable by setting it equal to this value. (E.g., `priorcell = min(insertion, deletion, substitution)`)
  - Finally, the loop needs to append this new cell value (now going under the alias `priorcell`) to the `row` list.
5. This *for*-loop will populate all the values in the row. The last thing to do is return the row, outside the *for*-loop.

Write your function in the code block below.

In [10]:
def nextrow(prior_row, word1, letter):
  row = []
  row.append(prior_row[0] + 1)
  prior_cell = row[0]

  for index in range(len(word1)):
    substitution = prior_row[index] + substitution_penalty(word1[index] , letter)
    insertion = prior_row[index + 1] + 1
    deletion = prior_cell + 1

    prior_cell = min(insertion, deletion, substitution)
    row.append(prior_cell)

  return row

If the function is working, the code block below should give you the following outputs:
```
[1, 0, 1, 2, 3, 4]
[2, 1, 2]
[2, 1, 2, 1, 2, 3, 4]
```

In [11]:
print(nextrow([0, 1, 2, 3, 4, 5], 'IDAHO','I'))
print(nextrow([1, 2, 1], 'IT', 'I'))
print(nextrow([1, 0, 1, 2, 3, 4, 5], 'TERROR', 'R'))

[1, 0, 1, 2, 3, 4]
[2, 1, 2]
[2, 1, 2, 1, 2, 3, 4]


# 5 Making the entire table
Now we can define the `minedit()` function, building an entire minimum edit distance table. The prep work we've done, building component functions, considerably simplifies things. 
1. Write a function labeled `minedit` that takes two words `word1` and `word2` as input. 
2. Within that function, use some label (like `m` as in `m=len(word1)`) to stand for the length of `word1`, and use another label (like `n`) to stand for the length of `word2`.
3. Create the first row of the table with `firstrow(m)`. That creates a row incrementing from 0 for each letter in `word1`. Give this first row the label `priorrow`: 
```
priorrow = firstrow(m)
```
4. Then create a list called `table` containing the first row (under the alias `priorrow` for now).
5. You then need a *for*-loop that iterates over `range(n)`, the number of letters in `word2`. For each `i` in that range, create a new row using the `nextrow()` function. Then append that row to `table`, and update `priorrow` to be this new row.
6. This *for*-loop will generate all the rows for the table. The last thing to do is just return the output list `table`.

In [12]:
def minedit(word1, word2):
  m = len(word1) # input word
  n = len(word2) # correct word

  prior_row = firstrow(m)
  table = []
  table.append(prior_row)

  for index in range(n):
    next_row = nextrow(prior_row, word1, word2[index])
    table.append(next_row)
    prior_row = next_row
  
  return table

When you're done, try out the code block below.

In [14]:
# print_table(minedit("IDAHO","IDOL"))
# print_table(minedit("IT","TI"))
# print_table(minedit("SHRIEK","GHOST"))
# print_table(minedit("TERROR","TRERO"))


# This is my answer for Skills Day 1 Question 13
# print_table(minedit("LINGUISTICS", "LOGICIAN"))
print_table(minedit("RAY", "RO" ))

0	1	2	3	
1	0	1	2	
2	1	2	3	



Next time, we will work on using tables like this to create an automated typo-correction system. 

If you finish early here, there are a few things that would be useful to work on.
1. Try 'beautifying' the table output. Include letter labels for the columns and rows, using the symbol `'#'` for the beginnings of words, like we saw in the slides.
2. There are lots of ways to optimize the algorithm above. If you can think of anything, let me know; I'll give you a free point in a skill of your choosing.
3. Think about how you could modify the `substitution_penalty()` function to be sensitive to the *likelihood* of different substitutions. For instance, two letters are more likely to be substituted if they are closer on a keyboard. Substituting *R* for *T* should incur less of a penalty than substituting *P* for *Z*. If you want, I can give you a list of distances between keys.
4. Think about bringing transpositions into this system (that's like swapping *L* and *I* in *SILD* for *SLID*, or *R* and *N* in *SPNIRT* for *SPRINT*). What kind of information would you need to calculate the cost of a transposition error? Where would you call such a function in the algorithm above?
